To run code:

In [1]:
import numpy as np
import findspark
findspark.init()
findspark.find()
import pyspark
import pyspark
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
import time
import sys
import timeit

In [2]:
conf = pyspark.SparkConf()
sc = pyspark.SparkContext.getOrCreate()
#spark.conf.set("spark.executor.instances", 4)
#spark.conf.set("spark.executor.cores", 4)
#pyspark.SparkContext.setSystemProperty('spark.executor.memory', '6g')
#pyspark.SparkContext.setSystemProperty('spark.driver.memory', '14g')
#pyspark.SparkContext.setDeployMode('client')
#sc = pyspark.SparkContext('spark://192.168.2.84:7077','Run experiment')
spark = SparkSession.builder.getOrCreate()

In [4]:
string_result = '1110100000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000'
print(sys.getsizeof(string_result))

1073


In [5]:
int_encoding = int(string_result,2)
print(sys.getsizeof(int_encoding))

164


In [6]:
def encode(seq):
    bitlist = list(seq)
    out = []
    i=0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | int(bit)
        i+=1
    return out

list_encoding = encode(string_result)
print(sys.getsizeof(list_encoding))

344


In [6]:
RDD_4mer = sc.textFile("/kmer_counting/output_java_spark_4mer/part-00000")

# # Most Frequence 4mers
# embedding_kmers = [str(sub.split(',')[0][1:]) for sub in RDD_4mer.take(1024)]
# embedding_kmers[:10]

kmer_seed = 77

# randomly generated most frequent 1024 4mers from the top 3000 4mers
#embedding_kmers = np.array([str(sub.split(',')[0][1:]) for sub in RDD_4mer.take(3000)])
#embedding_kmers_idxs =  np.random.default_rng(seed=kmer_seed).choice(range(3000),size=1024,replace=False)
#print(embedding_kmers_idxs)
#embedding_kmers = list(embedding_kmers[embedding_kmers_idxs])
#embedding_kmers[:10]

# # randomly generated most frequent 1024 4mers from the top 4000 4mers
embedding_kmers = np.array([str(sub.split(',')[0][1:]) for sub in RDD_4mer.take(4000)])
embedding_kmers_idxs =  np.random.default_rng(seed=kmer_seed).choice(range(4000),size=1024,replace=False)
print(embedding_kmers_idxs)
embedding_kmers = list(embedding_kmers[embedding_kmers_idxs])
embedding_kmers[:10]

[3788  957 1642 ... 3131 1747 1687]


['GELR',
 'GAAP',
 'TLVL',
 'GVGG',
 'LDIL',
 'SSLT',
 'ASSA',
 'EAAR',
 'LGVG',
 'AILS']

In [7]:
test = 'AAAAEILALEILA'

def int_encoding(seq):
    return (int("".join(["1" if kmer in seq else "0" for kmer in embedding_kmers]),2),1)

def encode(seq):
    bitlist = [1 if kmer in seq else 0 for kmer in embedding_kmers]
    out = []
    i=0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | bit
        i+=1
    return tuple(out)

print(timeit.timeit(lambda: int_encoding(test), number=10000))
print(timeit.timeit(lambda: encode(test), number=10000))

0.6520411821547896
2.0138762881979346


In [13]:
seed = np.random.randint(1,1000)
print("Seed: "+str(seed))

def test_random_sequences():
    random_sequences_idxs = np.random.default_rng(seed=seed).choice(range(118827660),size=100000,replace=False)
    print("Counted examples test_random_sequences: "+str(textRDD.zipWithIndex().filter(lambda pair: pair[1] in random_sequences_idxs).count()))

def test_sample():
    print("Counted examples test_sample: "+str(sc.parallelize(textRDD.sample(False, (100500/118827660), seed=seed).take(100000)).count()))

def test_takeSample():
    print("Counted examples test_takeSample: "+str(sc.parallelize(textRDD.takeSample(False, (100000), seed=seed)).count()))
running_sequences = 1
running_samples = 5

print(str(running_sequences) + " run of test_random_sequences took: "
      +str(timeit.timeit(lambda: test_random_sequences(), number=running_sequences)))
print(str(running_samples) + " runs of test_sample took: "
      +str(timeit.timeit(lambda: test_sample(), number=running_samples)))
print(str(running_samples) + " runs of test_takeSample took: "
      +str(timeit.timeit(lambda: test_takeSample(), number=running_samples)))

Seed: 111
Counted examples test_random_sequences: 100000
1 run of test_random_sequences took: 770.4607992549427
Counted examples test_sample: 99953
Counted examples test_sample: 99953
Counted examples test_sample: 99953
Counted examples test_sample: 99953
Counted examples test_sample: 99953
5 runs of test_sample took: 521.1825094870292
Counted examples test_takeSample: 100000
Counted examples test_takeSample: 100000
Counted examples test_takeSample: 100000
Counted examples test_takeSample: 100000
Counted examples test_takeSample: 100000
5 runs of test_takeSample took: 976.9337997338735


In [9]:
def count_collisions(kmers,seed):
    def encoding(seq):
        return (int("".join(["1" if kmer in seq else "0" for kmer in kmers]),2),1)
    sample = sc.parallelize(textRDD.sample(False, (100500/118827660), seed=seed).take(100000))
    return sample.map(encoding).reduceByKey(lambda a, b: a+b).values().filter(lambda x: x>1).map(lambda x: x-1).sum()

In [3]:
import random
def encoding(kmers,seq):
        return (int("".join(["1" if kmer in seq else "0" for kmer in kmers]),2),1)
splits = 256
split_size = 4096//splits
rdd_4mer = sc.textFile("/kmer_counting/output_java_spark_4mer/part-00000").map(lambda x: str(x.split(',')[0][1:]))
rdd_4096 = rdd_4mer.take(4096)
random.Random(77).shuffle(rdd_4096)
kmer_list = [rdd_4096[i*split_size:(i+1)*split_size] for i in range(splits)]
num_tries = 5
sample_count = 100000
#samples = sc.parallelize(textRDD.sample(False, ((sample_count+500)/118827660), seed=77).take(sample_count))
textRDD = sc.textFile("/data/linearized_squence.fasta").filter(lambda x: not x.startswith('>'))
best_kmers = []
for i in range(1024//split_size):
    print("Determine "+str((i+1)*split_size)+" kmers")
    samples = sc.parallelize(textRDD.sample(False, ((sample_count+500)/118827660)).take(sample_count))
    collisions = []
    for j in range(len(kmer_list)):
        collisions.append(1-samples.map(lambda seq: encoding(best_kmers+kmer_list[j],seq))\
                          .reduceByKey(lambda a, b: a+b).values().filter(lambda x: x>1)\
                          .map(lambda x: x-1).sum()/sample_count)
    f = lambda i: collisions[i]
    arg_max = max(range(len(collisions)), key=f)
    print("Minimal collisions: "+str(collisions[arg_max]))
    best_kmers += kmer_list[arg_max]
    with open("greedy_collision.txt","a") as f:
        f.write("Determine "+str((i+1)*split_size)+" kmers\n")
        f.write("Minimal collisions: "+str(collisions[arg_max])+"\n")
    with open("best_kmers_test.txt","a") as f:
        for k in kmer_list[arg_max]:
            f.write(k+"\n")

Determine 16 kmers
Minimal collisions: 0.006709999999999994
Determine 32 kmers
Minimal collisions: 0.03702000000000005
Determine 48 kmers
Minimal collisions: 0.08553
Determine 64 kmers
Minimal collisions: 0.14171
Determine 80 kmers
Minimal collisions: 0.20550999999999997
Determine 96 kmers
Minimal collisions: 0.26636000000000004
Determine 112 kmers
Minimal collisions: 0.32341
Determine 128 kmers
Minimal collisions: 0.37971
Determine 144 kmers
Minimal collisions: 0.42757999999999996
Determine 160 kmers
Minimal collisions: 0.47438
Determine 176 kmers
Minimal collisions: 0.5166
Determine 192 kmers
Minimal collisions: 0.55725
Determine 208 kmers
Minimal collisions: 0.5913900000000001
Determine 224 kmers
Minimal collisions: 0.61862
Determine 240 kmers
Minimal collisions: 0.6496
Determine 256 kmers
Minimal collisions: 0.67696
Determine 272 kmers
Minimal collisions: 0.69634
Determine 288 kmers
Minimal collisions: 0.72069
Determine 304 kmers
Minimal collisions: 0.73744
Determine 320 kmers
Mini

In [4]:
best_kmers = []
with open('best_kmers.txt','r') as f:
    for line in f:
        best_kmers.append(line.strip())

best_kmers_unshuffled = []
with open('best_kmers_unshuffled.txt','r') as f:
    for line in f:
        best_kmers_unshuffled.append(line.strip())
        
best_kmers_test = []
with open('best_kmers_test.txt','r') as f:
    for line in f:
        best_kmers_test.append(line.strip())


def encode(kmers,seq):
    return (int("".join(["1" if kmer in seq else "0" for kmer in kmers]),2),1)

def create_exp(seed,kmers):
    return sc.parallelize(textRDD.sample(False, (100500/118827660), seed=seed)\
                          .take(100000)).map(lambda seq: encode(kmers,seq))

def count_collisions(seq_rdd):
    collisions = seq_rdd.reduceByKey(lambda a, b: a+b).values()\
    .filter(lambda x: x>1).map(lambda x: x-1).sum()
    return collisions

In [6]:
textRDD = sc.textFile("/data/linearized_squence.fasta").filter(lambda x: not x.startswith('>'))
start_time = time.time()
collision_list = []
collision_file_name = "collision_count_test"
i = 0
for i in range(1,101):
    collisions = count_collisions(create_exp(i,best_kmers_test))
    collision_list.append(collisions)
    print("Finished example "+str(i))
    print("Collisions for example "+str(i)+": "+str(100*collisions/100000)+"%")
sc.stop()

needed_time = (time.time() - start_time)/60

print(str(i)+" jobs took "+str(needed_time)+" minutes.")
print(collision_list)
with open(collision_file_name,'w') as f:
    f.write("Time took for "+str(i)+" jobs: "+str(needed_time)+" minutes.\n")
    for s in collision_list:
        f.write(str(s)+'\n')
    f.write("Average collisions: " + str(sum(collision_list)/len(collision_list)))

Finished example 1
Collisions for example 1: 2.684%
Finished example 2
Collisions for example 2: 2.655%
Finished example 3
Collisions for example 3: 2.701%
Finished example 4
Collisions for example 4: 2.69%
Finished example 5
Collisions for example 5: 2.637%
Finished example 6
Collisions for example 6: 2.619%
Finished example 7
Collisions for example 7: 2.636%
Finished example 8
Collisions for example 8: 2.613%
Finished example 9
Collisions for example 9: 2.678%
Finished example 10
Collisions for example 10: 2.686%
Finished example 11
Collisions for example 11: 2.681%
Finished example 12
Collisions for example 12: 2.619%
Finished example 13
Collisions for example 13: 2.55%
Finished example 14
Collisions for example 14: 2.706%
Finished example 15
Collisions for example 15: 2.773%
Finished example 16
Collisions for example 16: 2.736%
Finished example 17
Collisions for example 17: 2.771%
Finished example 18
Collisions for example 18: 2.761%
Finished example 19
Collisions for example 19: 2

In [10]:
def encode(seq):
    bitlist = [1 if kmer in seq else 0 for kmer in embedding_kmers]
    out = []
    i=0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | bit
        i+=1
    return out

def create_exp(seed):

    random_sequences_idxs = np.random.default_rng(seed=seed).choice(range(118827660),size=100000,replace=False)
#     np.random.seed(seed)
#     random_sequences_idxs = np.random.choice(range(118827660),size=100000,replace=False)

#     # It may lead to same sequences being selected
#     random_sequences_idxs = np.random.randint(0, 10, 1, dtype=int)
#     print(random_sequences_idxs)

#     rdd_ids = sc.parallelize(random_sequences_idxs).coalesce(1)
#     rdd_ids.saveAsTextFile("/test_encodings/exp_"+str(seed)+"/id")

    seq_rdd = textRDD.zipWithIndex().filter(lambda pair: pair[1] in random_sequences_idxs)\
    .map(lambda pair: encode(pair[0]))
#     seq_rdd.saveAsTextFile("/test_encodings/exp_" + str(seed)+"/encoding")
    return seq_rdd

def count_collisions(seq_rdd):
    #textRDD = sc.textFile("/test_encodings/exp_"+str(seed)+"/encoding/part-*")
    collisions = seq_rdd.map(lambda x: tuple(x)).map(lambda x: (x,1)).reduceByKey(lambda a,b:a +b)\
    .values().filter(lambda x: x>1).map(lambda x: x-1).sum()
    return collisions

In [ ]:
textRDD = sc.textFile("/data/linearized_squence.fasta").filter(lambda x: not x.startswith('>'))
start_time = time.time()
collision_list = []
i = 0

for i in range(1,101):
    collisions = count_collisions(create_exp(i))
    collision_list.append(collisions)
    print("Finished example "+str(i))
    print("Collisions for example "+str(i)+": "+str(collisions))
sc.stop()

needed_time = (time.time() - start_time)/60

print(str(i)+" jobs took "+str(needed_time)+" minutes.")
print(collision_list)
with open("collision_count",'w') as f:
    f.write("Time took for "+str(i)+" jobs: "+str(needed_time)+" minutes.\n")
    for s in collision_list:
        f.write(str(s)+'\n')
    f.write("Average Collision: " + str(sum(collision_list)/len(collision_list)))

Finished example 1
Collisions for example 1: 3523
Finished example 2
Collisions for example 2: 3451
Finished example 3
Collisions for example 3: 3416
Finished example 4
Collisions for example 4: 3417
